Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## Define Strategy

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

In [ ]:
class SimpleStrategy(Strategy):
    def __init__(self, asset, quantity):
        super().__init__()
        self.asset = asset
        self.quantity = quantity

    def log_all(self, orders, data, ctx, utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(utc)
            self.log_ohlcv(data, self.asset.symbol, ctx.exchange.id)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        quantity = .05
        price = data.get('close', self.asset.symbol, ctx.exchange.id)
        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, self.asset, quantity, price)
            orders.append(order)
        else:
            order = order_manager.build_market_sell_order(
                ctx.exchange, self.asset, quantity)
            orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)
        self.log_all(orders, data, ctx, data.get('utc'))
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

### Backtest From Local CSV Files

In [ ]:
# Download some data from Binance
exchange_id = ex_cfg.BINANCE
timeframe = Timeframe.THIRTY_MIN
start = datetime.datetime(year=2018, month=1, day=1, hour=0)
end = datetime.datetime(year=2018, month=1, day=2, hour=0)
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.BTC, coins.USDT)]
data_exchange = load_exchange(exchange_id)
ohlcv_feed.download_ohlcv([data_exchange], assets, timeframe, start, end)

In [ ]:
# Setup the Feed Based Paper Exchange
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=cash_currency,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=timeframe
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = OHLCVFileFeed(
    exchange_ids=[exchange_id],
    assets=assets,
    timeframe=timeframe,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed, exchange_id)
strategy = SimpleStrategy(assets[0], quantity=.05)
experiment_name = 'default_backtest'

In [ ]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

In [ ]:
record

### Backtest from External Exchange Data

In [ ]:
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=cash_currency,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, ex_cfg.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow()-datetime.timedelta(days=1),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT
Downloaded rows: 500


In [ ]:
experiment_name = 'default_backtest_exchange'
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT
Downloaded rows: 500


---------------------------------------
Epoch 1 - Timestep: 2018-01-22T04:02:00
---------------------------------------
OHLCV
    O: 0.0914 | C: 0.0915 | V: 103.3 | T: 2018-01-22T04:02:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0915 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00


Equal? True
Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT


---------------------------------------
Epoch 2 - Timestep: 2018-01-22T04:03:00
---------------------------------------
OHLCV
    O: 0.0914 | C: 0.0914 | V: 127.6 | T: 2018-01-22T04:03:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9954 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.995426, 'used': 0.0, 'total': 0.995426}
    ETH - {'free': 0.05, 'used': 0.0, 'total': 0.05}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.09147999999999999, 'latest_price': 0.09147999999999999}
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 500
Equal? True
Downloading: ETH/BTC
Downloaded rows: 500
Downloading: LTC/BTC
Downloaded rows: 500
Downloading: BTC/USDT


...

---------------------------------------
Epoch 15 - Timestep: 2018-01-22T04:16:00
---------------------------------------
OHLCV
    O: 0.0915 | C: 0.0915 | V: 138.2 | T: 2018-01-22T04:16:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0915 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0001 Total Val: 1.0001 PnL: 0.0001 Returns: 0.0001
BALANCE
    BTC - {'free': 1.00008475, 'used': 0.0, 'total': 1.00008475}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.0, 'cost_price': 0.0, 'latest_price': 0.091401}
METRICS
    SMA: 5.00
    RSI: 10.00


Order Failed {
    "id": "0be2e90b5e9049a5bf23603c5d4e5fff",
    "exchange_id": "binance",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-23T04:01:22.736388",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "attempts": 3,
    "trades": [],
    "error": {
        "code": "INSUFFICIENT_FUNDS",
        "msg": "Insufficient funds to place order for asset ETH/BTC of cost: 0.00459. BTC balance is: 1.00008475. ETH balance is 0.0"
    },
    "symbol": "ETH/BTC"
}
Order Failed {
    "id": "9273342e85bf49d0be2679fc275618ed",
    "exchange_id": "binance",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-23T04:01:23.781393",
    "opened_time": null,
    "filled_time": null,
    "ca

---------------------------------------
Epoch 16 - Timestep: 2018-01-22T04:17:00
---------------------------------------
OHLCV
    O: 0.0914 | C: 0.0914 | V: 349.9 | T: 2018-01-22T04:17:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9955 Total Val: 1.0001 PnL: 0.0001 Returns: 0.0001
BALANCE
    BTC - {'free': 0.9955116500000001, 'used': 0.0, 'total': 0.9955116500000001}
    ETH - {'free': 0.05, 'used': 0.0, 'total': 0.05}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.091462, 'latest_price': 0.091462}
METRICS
    SMA: 5.00
    RSI: 10.00


Order Failed {
    "id": "611876cbdc324ff3a29d0caeb5cb78b0",
    "exchange_id": "binance",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-23T04:01:24.847618",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "attempts": 2,
    "trades": [],
    "error": {
        "code": "INSUFFICIENT_FUNDS",
        "msg": "Insufficient funds to place order for asset ETH/BTC of cost: 0.00459. BTC balance is: 1.0001055. ETH balance is 0.0"
    },
    "symbol": "ETH/BTC"
}
Order Failed {
    "id": "d7d6e034b4954af48cf5ef654234c7af",
    "exchange_id": "binance",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-23T04:01:27.249464",
    "opened_time": null,
    "filled_time": null,
    "can

---------------------------------------
Epoch 17 - Timestep: 2018-01-22T04:18:00
---------------------------------------
OHLCV
    O: 0.0914 | C: 0.0914 | V: 202.0 | T: 2018-01-22T04:18:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0001 Total Val: 1.0001 PnL: 0.0001 Returns: 0.0001
BALANCE
    BTC - {'free': 1.0001055, 'used': 0.0, 'total': 1.0001055}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.0, 'cost_price': 0.0, 'latest_price': 0.091437}
METRICS
    SMA: 5.00
    RSI: 10.00


## Simulate Orders

In [ ]:
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=coins.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, ex_cfg.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

In [ ]:
experiment_name = 'default_simulate'
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
exchange = load_exchange(ex_cfg.BINANCE)
cash = exchange.fetch_balance().get(coins.BTC)[BalanceType.FREE]
cash_currency = coins.BTC
print(coins.BTC, "cash", cash)

In [ ]:
balance = Balance(
    cash_currency=coins.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=None,
    end=None
)
experiment_name = 'default_live'

In [ ]:
runner.live(experiment_name, exchange, balance, portfolio, feed, strategy)